<a href="https://colab.research.google.com/github/ssp22/485Design-Project/blob/master/optimization_final_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [27]:
# Define parameters
  dP = 40 # bar
  dPi = 5.7 # bar
  R = 0.1 # at 40 bar, 5C
  k_w = 1.25 * 10 ** (-5) # g / (cm^2 s bar)
  diff_P = dP - dPi
  X = 0.05 # split ratio
  C_Et0 = 0.0395 # weight percent
  # M_f = 1000 # g / s # flow out of tank
  t = np.linspace(0,10000,10000)
  Product = 29.8 * 3968 * 6 * 10**4
  Max = Product
  A = 953000 # cm^2 # membrane area
  Parameters = [1,2,3,4,5]

for i in range(1,20):
    X = i / 100
    for j in range(1900000,2100000,1000):
        m_0 = j # g # initial batch feed
        for k in range(800,1500,100):
          M_f = k
          # solve ODE
          C = odeint(dCdt2,C_Et0,t)
          count = 0
          val= 10
          if (np.amin(C)<=0.00395):
            while (val >= 0.00395):
              val = C[count]
              count += 1
            # break
          if (val <= 0.00395):
            Total = 0.8*365*20*3600/t[count]*m_0
          if (Total > Max):
            Max = Total
            Parameters[0] = X
            Parameters[1] = m_0
            Parameters[2] = M_f
            Parameters[3] = t[count]
            Parameters[4] = val
 

print('split ratio = ', Parameters[0] )
print('batch feed = ',Parameters[1],' g')
print('flow out of tank = ',Parameters[2],'g/s')
print('time for one batch',Parameters[3] ,'s')
print('final conc = ', Parameters[4])
print('total product= ', Max,' g')

# optimization result:
# split ratio =  0.01
# batch feed =  2083000  g
# flow out of tank =  900 g/s
# time for one batch 6166.616661666166 s
# final conc =  [0.00394997]
# total product=  7101623856.762895  g

split ratio =  0.01
batch feed =  2083000  g
flow out of tank =  900 g/s
time for one batch 6166.616661666166 s
final conc =  [0.00394997]
total product=  7101623856.762895  g


In [0]:
def dCdt2(C_Et,t):
    # Define flow rates
    M_p1 = A * k_w * diff_P * (1 + C_Et * (1-R)) 
    M_r1 = M_f - M_p1
    if (M_r1 < 0):
      return 0
    C_r1 = (M_f * C_Et - M_p1 * (1-R) * C_Et) / M_r1
    M_p2 = A * k_w * diff_P * (1 + C_r1 * (1-R))
    M_r2 = (1-X) * M_r1 - M_p2
    if (M_r2 < 0):
      return 0
    C_r2 = ((1-X)*M_r1*C_r1 - M_p2 * (1-R) * C_r1) / M_r2
    dCdt = (M_r1 * X * C_r1 + M_r2 * C_r2 - M_f * C_Et) / m_0
    M_water = M_p1 + M_p2
    return dCdt